# NoSQL: Basic queries on non-relational databases

## NoSQL: Review

A NoSQL database is designed to handle data that are not structured in tabular relations. The data can be stored in key-value pairs (like Python dictionaries), documents (JSON), or graphs.

<img src="../assets/nosql_vs_sql.jpeg"  width = 500 px></img>

Unlike relational databases (SQL), NoSQL does not (necessarily) employ tables and rows. Also they are much more flexible since the structure of the entries is not predefined. Therefore you can add data without defining anything (even not the table itself).

[This article](https://www.integrate.io/blog/the-sql-vs-nosql-difference/) gives a recap of the biggest differences between SQL and NoSQL database systems:
|SQL|NoSQL|
|---|---|
|SQL databases are relational| NoSQL databases are non-relational. In NoSQL you cannot make relationships between tables (`JOIN`)|
|SQL databases are table-based |NoSQL databases are document, key-value, graph, or wide-column stores.|
|SQL databases use structured query language and have a predefined schema.| NoSQL databases have dynamic schemas for unstructured data.|
|SQL databases are vertically scalable (by adding processing power)|NoSQL databases are horizontally scalable (by adding servers/machines).|
|SQL databases are better for multi-row transactions|NoSQL is better for unstructured data like documents or JSON.|

In the image, below you will find examples of popular SQL and NoSQL databases. 

<img src="../assets/popular_examples_nosql_sql.jpeg" width =500px></img>

In the next section, we'll cover an example using MongoDB. You will see that we can translate many of the basic SQL queries you know.

## MongoDB

<img src="https://upload.wikimedia.org/wikipedia/fr/thumb/4/45/MongoDB-Logo.svg/527px-MongoDB-Logo.svg.png" />

A lot of companies provide NoSQL architectures. One of the most popular is [MongoDB](https://www.mongodb.com/). A MongoDB database contains collections (tables) of documents (entries). The documents are stored in the JSON format which is very convenient to handle with Python!

The syntax of the queries is based on JavaScript. They basically look like python dictionaries.

Before diving in some exercises, you can have a look to [this quick intro](https://www.mongodb.com/docs/manual/tutorial/query-documents/) about MongoDB queries.

### Creating the database

We have created and filled a MongoDB database for you. You probably know already the database. It is the list of country leaders that you have used already in the Wikipedia project.

You can set it up by deploying the Docker image we have pre-built:

In [1]:
# You can run it from this notebook with:
!docker-compose up -d


# Or in your terminal with:
# docker-compose up -d

/bin/bash: line 1: docker-compose: command not found


### Conneting to the database

For using MongoDB through Python you will need to install the `pymongo` library.

In [5]:
from pymongo import MongoClient

# Creation of a MongoDB Client (by giving the host and the port)
client = MongoClient(host="localhost", port=27017)

# Instantiation of the database
db = client["becode"]

# Let's see which collections are in the database
db.list_collection_names()

['leaders']

### Basic Queries

**1. Show the first leader of the `leaders` collection by using the `find_one` method**

The corresponding `SQL` query would be:

```sql
SELECT * FROM leaders LIMIT 1;
```

In [6]:
db["leaders"].find_one()

{'_id': ObjectId('64da27087c3cf3ff960cd957'),
 'id': 'Q7747',
 'first_name': 'Vladimir',
 'last_name': 'Putin',
 'birth_date': '1952-10-07',
 'death_date': None,
 'place_of_birth': 'Saint Petersburg',
 'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87',
 'start_mandate': '2000-05-07',
 'end_mandate': '2008-05-07',
 'country': 'ru'}

**2. Show the first leader of the collection whose country is Belgium**

To that purpose we will use the query as first parameter of the `find_one` function. The query is formatted as a dictionary.

In SQL it would be like adding a `WHERE` condition to the query:

```sql
SELECT * FROM leaders WHERE country = 'be' LIMIT 1;
```

In [7]:
db["leaders"].find_one({"country": "be"})

{'_id': ObjectId('64da27087c3cf3ff960cd986'),
 'id': 'Q12981',
 'first_name': 'Yves',
 'last_name': 'Leterme',
 'birth_date': '1960-10-06',
 'death_date': None,
 'place_of_birth': 'Wervik',
 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
 'start_mandate': '2009-11-25',
 'end_mandate': '2011-12-06',
 'country': 'be'}

**3. Select some fields to display**

Let's use the same query by displaying only the `first_name` and the `last_name` of the leader. It corresponds to a `SELECT` in SQL.

We will use the [project](https://www.mongodb.com/docs/manual/tutorial/project-fields-from-query-results/) as the second parameter of the function. It is also formatted as a dictionary whose the key contains the targeted field and the value is `1` if we want the field to be displayed

The corresponding SQL query would be:

```sql
SELECT first_name, last_name FROM leaders WHERE country = 'be' LIMIT 1;
```

In [8]:
db["leaders"].find_one({"country": "be"}, {"first_name": 1, "last_name": 1})

{'_id': ObjectId('64da27087c3cf3ff960cd986'),
 'first_name': 'Yves',
 'last_name': 'Leterme'}

We can also decide to not display a field. In that case we put `0` as value for the dictionary.

In [9]:
db["leaders"].find_one({"country": "be"}, {"wikipedia_url": 0, "id": 0})

{'_id': ObjectId('64da27087c3cf3ff960cd986'),
 'first_name': 'Yves',
 'last_name': 'Leterme',
 'birth_date': '1960-10-06',
 'death_date': None,
 'place_of_birth': 'Wervik',
 'start_mandate': '2009-11-25',
 'end_mandate': '2011-12-06',
 'country': 'be'}

**4. Find the distinct countries**

The SQL equivalent:

```sql
SELECT DISTINCT country FROM leaders;
```

In [10]:
db["leaders"].find().distinct("country")

['be', 'fr', 'ma', 'ru', 'us']

**5. Find all the leaders who are still alive**

We can assume that they have no `death_date`, isn't?

In [12]:
db["leaders"].find({"death_date": None}, {"last_name":1, "first_name":1, "country":1})

As you see the `find` command returns a cursor. No worries we can process it by using a simple Python loop!

In [13]:
for leader in db["leaders"].find({"death_date": None}, {"last_name":1, "first_name":1, "country":1}):
    print(f"{leader['first_name']} {leader['last_name']} ({leader['country']})")

Vladimir Putin (ru)
Mohammed None (ma)
Dmitry Medvedev (ru)
Mohammed None (ma)
Mohammed None (ma)
Barack Obama (us)
Joe Biden (us)
Bill Clinton (us)
George Bush (us)
Donald Trump (us)
Jimmy Carter (us)
Yves Leterme (be)
Herman Van Rompaey (be)
Guy Verhofstadt (be)
Elio Di Rupo (be)
Alexander De Croo (be)
Mark Eyskens (be)
Sophie Wilmès (be)
François Hollande (fr)
Nicolas Sarkozy (fr)
Charles Michel (be)
Emmanuel Macron (fr)


**6. Let's now insert the leader of tomorrow: you?**

As you know, MongoDB is flexible. It means that you can add entries although some fields are missing. Let's give a try:

In [14]:
you = {
    'first_name': 'ADD HERE YOUR FIRST NAME',
    'last_name': 'ADD HERE YOUR LAST NAME',
    'birth_date': 'ADD HERE YOUR BIRTH DATE',
    'country': 'ADD HERE YOUR COUNTRY CODE'
}
db["leaders"].insert_one(you)

Let's have a look to your data!

In [15]:
db["leaders"].find_one({"first_name": "ADD HERE YOUR FIRST NAME"})

{'_id': ObjectId('64da27ea17ab33d0790aedb2'),
 'first_name': 'ADD HERE YOUR FIRST NAME',
 'last_name': 'ADD HERE YOUR LAST NAME',
 'birth_date': 'ADD HERE YOUR BIRTH DATE',
 'country': 'ADD HERE YOUR COUNTRY CODE'}

We can observe two things here:
- A field `_id` has been automatically added by Mongo. This field is incremental. That means that it will always be higher than the previous element of the collection.
- Some fields are missing (the `place_of_birth` for instance). This is a property of NoSQL. All fields are not mandatory!

We can check missing values by using the query `{"$exists":False}`:

In [16]:
for leader in db["leaders"].find({"place_of_birth":{"$exists":False}}):
    print(leader)

{'_id': ObjectId('64da27ea17ab33d0790aedb2'), 'first_name': 'ADD HERE YOUR FIRST NAME', 'last_name': 'ADD HERE YOUR LAST NAME', 'birth_date': 'ADD HERE YOUR BIRTH DATE', 'country': 'ADD HERE YOUR COUNTRY CODE'}


### Update data

Since your place of birth is missing in the data, let's add it now. The update function has two main arguments:
- a query that will select the entries to update
- an update operation. As always, it is formatted as a dictionary

In [17]:
db["leaders"].update_one({"first_name": "ADD HERE YOUR FIRST NAME"}, {"$set": {"place_of_birth": "ADD HERE YOUR PLACE OF BIRTH"}})

#### Remove data

Your dream is over ;-) Since you will not be a leader of tomorrow, we will remove you from the collection. The `delete` (or `delete_one`) function has one main argument: the query that will select the entries to be removed.

In [18]:
db["leaders"].delete_one({"first_name": "ADD HERE YOUR FIRST NAME"})

## Your Turn!

Based on your knowledge and some Google search try to create the following queries:

- Remove the leaders who have an empty or null (`None`) last name
- Display all unique first names from the collection
- Transform all the dates of the dataset by a datetime object (they are currently strings which is not a good practice). You can use a python script that interacts with the DB instead of doing everything in a single query
- Display the 10 older leaders ordered by their birth date (search for how to sort and to use limits in MongoDB)
- Create a Python script that computes the numbers of leaders by country
- Do the same by using a MongoDB [aggregation pipeline](https://www.mongodb.com/docs/manual/aggregation/)

In [ ]:
# Your code here (feel free to add some extra code blocks!)

## Resources:
* [NoSQL Concepts (DataCamp)](https://www.datacamp.com/courses/nosql-concepts)
* [Introduction to MongoDB using Python (DataCamp)](https://www.datacamp.com/courses/introduction-to-using-mongodb-for-data-science-with-python)
* [Getting started with MongoDB](https://docs.mongodb.com/manual/tutorial/)
* [Python MongoDB Tutorial](https://www.mongodb.com/blog/post/getting-started-with-python-and-mongodb)
* [Introduction to MongoDB Learning Path](https://learn.mongodb.com/learning-paths/introduction-to-mongodb)
* [Build an App With Python, Flask, and MongoDB to Track UFOs](https://www.mongodb.com/developer/languages/python/flask-app-ufo-tracking/)